In [4]:
import pandas as pd
import numpy as np
import difflib
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer 
netflix=pd.read_csv('movies.csv')
features=['genres','keywords','tagline','cast','director','overview']
for i in features:
    netflix[i]=netflix[i].fillna('')
all_features=netflix['cast']+' '+netflix['genres']+' '+' '+netflix['keywords']+' '+netflix['tagline']+netflix['director']+netflix['overview']
vect=TfidfVectorizer()
feature_vector=vect.fit_transform(all_features)
similarity = cosine_similarity(feature_vector)
all_titles=netflix['title'].tolist()
movie=input("Enter your movie name : ")
all_close_match=difflib.get_close_matches(movie,all_titles)
close_match=all_close_match[0]
index=netflix[netflix.title==close_match]['index'].values[0]
similarity_score=list(enumerate(similarity [index]))
sorted_movies=sorted(similarity_score, key=lambda x:x[1],reverse=True)
print("Movies you might like : ")
count=1
for i in sorted_movies:
    index=i[0]
    score=int(i[1]*100)
    title_from_data=netflix[netflix.index==index]['title'].values[0]
    if count<=5:
        print(str(count)+'. '+title_from_data)
        count+=1

Enter your movie name : Iron man
Movies you might like : 
1. Iron Man
2. Iron Man 2
3. Iron Man 3
4. Avengers: Age of Ultron
5. The Avengers


In [10]:
import pickle
similarity = cosine_similarity(feature_vector)
file=open("similarity.pkl","wb")
pickle.dump(similarity, file)
file.close()


array([[1.        , 0.05016606, 0.02145534, ..., 0.02635719, 0.03100322,
        0.00674257],
       [0.05016606, 1.        , 0.04328573, ..., 0.0510804 , 0.03277859,
        0.01630919],
       [0.02145534, 0.04328573, 1.        , ..., 0.02616905, 0.02235067,
        0.01449825],
       ...,
       [0.02635719, 0.0510804 , 0.02616905, ..., 1.        , 0.03661528,
        0.03789082],
       [0.03100322, 0.03277859, 0.02235067, ..., 0.03661528, 1.        ,
        0.03310653],
       [0.00674257, 0.01630919, 0.01449825, ..., 0.03789082, 0.03310653,
        1.        ]])

In [13]:
import streamlit as st
import pickle
import difflib
st.title("Movie suggestion page")
mov_list=pickle.load(open('movies.pkl','rb'))
all_mov_list=mov_list['title'].values
def suggest_movies(title):
     all_titles = mov_list['title'].tolist()
     movie = title
     all_close_match = difflib.get_close_matches(movie, all_titles)
     close_match = all_close_match[0]
     index = mov_list[mov_list.title == close_match]['index'].values[0]
     similarity_score = list(enumerate(similarity[index]))
     sorted_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)
     count = 1
     suggestions=[]
     st.write("Here are few movies you might like :")
     for i in sorted_movies:
          index = i[0]
          title_from_data = mov_list[mov_list.index == index]['title'].values[0]
          if count <= 5:
               suggestions.append(title_from_data)
               count += 1
     return suggestions



similarity=pickle.load(open('similarity.pkl','rb'))


user_title = st.selectbox(
     'Hey there! please enter the title of your favorite movie : ',
     all_mov_list)

st.write('You selected:', user_title)
if st.button('Suggest'):
     for i in suggest_movies(user_title):
          st.write(i)